In [0]:
DEBUG = True

FIELD = 'draco'
PMAP_PATH = '/datascope/subaru/data/targeting/dSph/draco/pmap/draco_nb/pmap.h5'
OBS_PATH = '/datascope/subaru/data/targeting/dSph/draco/draco_tpall3e_g24.cat'
ISOCHRONES_PATH = '/datascope/subaru/data/cmdfit/isochrones/dartmouth/import/afep0_cfht_sdss_hsc'

In [0]:
import os, sys
from datetime import datetime
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib.gridspec import GridSpec
from matplotlib.colors import LinearSegmentedColormap
from matplotlib.patches import Ellipse, Circle
from scipy.special import logsumexp
from scipy.interpolate import interp1d

from astropy import wcs
from astropy import units as u
from astropy.coordinates import Angle

In [0]:
plt.rc('font', size=6) #controls default text size

In [0]:
%load_ext autoreload
%autoreload 2

In [0]:
if DEBUG and 'debug' not in globals():
    import debugpy
    debugpy.listen(('0.0.0.0', int(os.environ['PFS_TARGETING_DEBUGPORT'])))
    debug = True

# Initialize the plots

In [0]:
from pfs.ga.targeting.targets.dsph import GALAXIES
from pfs.ga.targeting.instrument import *
from pfs.ga.targeting.projection import WcsProjection
from pfs.ga.targeting.diagram import *

In [0]:
field = GALAXIES[FIELD]

hsc = field.get_photometry()
cmd = field.get_cmd()
ccd = field.get_ccd()

In [0]:
center = field.get_center()
pointings = field.get_pointings(SubaruPFI)
center, pointings

In [0]:
wcs = WcsProjection(center, proj='TAN')
wfc = SubaruWFC(center)
fov = FOV(projection=wcs)
pfi = SubaruPFI(instrument_options={'layout': 'calibration'})
fp = FP(wfc)

# Load the isochrones

In [0]:
from pfs.ga.isochrones.isogrid import IsoGrid
from pfs.ga.targeting import Isochrone

In [0]:
iso = IsoGrid()
iso.load(os.path.join(ISOCHRONES_PATH, 'isochrones.h5'))

In [0]:
iso.values.keys()

# Query the GAIA database

In [0]:
from pfs.ga.targeting.io import GaiaReader

In [0]:
reader = GaiaReader()
gaia = reader.cone_search((field.pos.ra, field.pos.dec), field.rad)
gaia.data

# Load the observations

In [0]:
r = field.get_text_observation_reader()
obs = r.read(OBS_PATH)
obs.data.head()

In [0]:
# Cross-match with GAIA

hsc_gaia_idx, separation = obs.cross_match(gaia)
print(f'GAIA->HSC source median separation: {np.median(separation.arcsec)} arcsec.')
hsc_gaia_mask = (separation.arcsec < 0.1)
print(f'HSC targets with matching GAIA targets: {hsc_gaia_mask.sum()}')

In [0]:
# Merge proper motions from GAIA into HSC

columns = ['parallax', 'pm', 'pmdec', 'pmra', 'err_parallax', 'err_pmdec', 'err_pmra']
obs.merge(gaia, hsc_gaia_idx, columns=columns, mask=hsc_gaia_mask)

In [0]:
obs.columns

# Load the pmap

In [0]:
from pfs.ga.targeting import ProbabilityMap, Isochrone

In [0]:
pmap = ProbabilityMap(cmd.axes)
pmap.load(PMAP_PATH)

In [0]:
# Bluest possible isochron from the CMD simulation
iso_blue = Isochrone()
iso_blue.from_isogrid(hsc, iso, Fe_H=-2.0, log_t=10.111, DM=19.2)

In [0]:
f, axs = plt.subplots(1, 2, figsize=(6, 4), dpi=120)

l0 = cmd.plot_probability_map(axs[0], pmap, 0)
# cmd.plot_isochrone(axs[0], iso_blue, observed=True, error_sigma=[-2.0, -2.0], c='blue', lw=0.5)
cmd.plot_isochrone(axs[0], iso_blue, observed=True, c='blue', lw=0.5)
axs[0].set_title("non-member")
axs[0].set_xlim(0, 2)
axs[0].grid()

l1 = cmd.plot_probability_map(axs[1], pmap, 1)
cmd.plot_isochrone(axs[1], iso_blue, observed=True, error_sigma=[-2.0, -2.0], c='blue', lw=0.5)
axs[1].set_title("member")
axs[1].set_xlim(0, 2)
axs[1].grid()

f.tight_layout()

# Select sample and assign priorities

In [0]:
from pfs.ga.targeting.selection import ProbabilityCut

In [0]:
obs.data.columns

In [0]:
# Apply priorities
probcut = ProbabilityCut(pmap, 1, lp_member_limit=np.log(0.001))
selection = field.get_selection_mask(obs, probcut)
field.assign_probabilities(obs, pmap, mask=selection)
field.assign_priorities(obs, isogrid=iso)

#field.assign_priorities_old(obs)

In [0]:
selection.sum(), np.isfinite(obs.data['p_member']).sum(), np.isnan(obs.data['p_member']).sum()

In [0]:
cmap = plt.get_cmap('tab10')

f = plt.figure(figsize=(8, 4), dpi=240)
gs = f.add_gridspec(2, 3, width_ratios=[2, 2, 3], height_ratios=[4, 1], wspace=0.4, hspace=0.5)

axs = [f.add_subplot(gs[0, 0]), f.add_subplot(gs[0, 1]), f.add_subplot(gs[0, 2], projection=wcs.wcs)]
cax = f.add_subplot(gs[1, :])

# cmd.plot_observation(axs[0], obs, c='gray', observed=True)
pp = np.sort(obs.data['priority'][obs.data['priority'] >= 0].unique())[::-1]
for p in pp:
    mask = (obs.data['priority'] == p)
    l = cmd.plot_observation(axs[0], obs, c=obs.data['priority'][mask], mask=mask, observed=True,
                             vmin=0, vmax=pp.max(), cmap=cmap)

    ccd.plot_observation(axs[1], obs, c=obs.data['priority'][mask], mask=mask, observed=True,
                             vmin=0, vmax=pp.max(), cmap=cmap)

    fov.plot_observation(axs[2], obs, c=obs.data['priority'][mask], mask=mask, observed=True,
                             vmin=0, vmax=pp.max(), cmap=cmap)

# xx = np.linspace(0, 1, 20)
# axs[0].plot(xx, 19.5 - 1.5 * xx)

# xx = np.linspace(0, 1, 20)
# axs[1].plot(xx, 0.25 * xx - 0.15)

# ccd.plot_observation(axs[1], obs, c='gray', observed=True)

# fov.plot_observation(axs[2], obs, c='gray', observed=True)
        
pp = field.get_pointings(SubaruPFI)
for p in pp:
    pfi.plot_focal_plane(axs[2], fov, corners=True, projection=SubaruWFC(p))

axs[2].set_xlim(0.75, -2.75)
axs[2].set_ylim(-2.75, 0.75)

f.suptitle('HSC targets')
f.colorbar(l, cax=cax, orientation='horizontal')

In [0]:
cmap = plt.get_cmap('tab10')

pp = np.sort(obs.data['priority'][obs.data['priority'] >= 0].unique())
for p in pp:

    f = plt.figure(figsize=(8, 4), dpi=240)
    gs = f.add_gridspec(1, 3, width_ratios=[2, 2, 3], wspace=0.4, hspace=0.5)

    axs = [f.add_subplot(gs[0, 0]), f.add_subplot(gs[0, 1]), f.add_subplot(gs[0, 2], projection=wcs.wcs)]

    cmd.plot_observation(axs[0], obs, c='lightgray', mask=selection, observed=True)
    ccd.plot_observation(axs[1], obs, c='lightgray', mask=selection, observed=True)

    mask = (obs.data['priority'] == p)
    l = cmd.plot_observation(axs[0], obs, c='blue', mask=mask, observed=True,
                             vmin=0, vmax=pp.max(), cmap=cmap)

    ccd.plot_observation(axs[1], obs, c='blue', mask=mask, observed=True,
                             vmin=0, vmax=pp.max(), cmap=cmap)

    fov.plot_observation(axs[2], obs, c='blue', mask=mask, observed=True,
                             vmin=0, vmax=pp.max(), cmap=cmap)


    # fov.plot_observation(axs[2], obs, c='gray', observed=True)
            
    for pointing in field.get_pointings(SubaruPFI):
        pfi.plot_focal_plane(axs[2], fov, corners=True, projection=SubaruWFC(pointing))

    axs[2].set_xlim(0.75, -2.75)
    axs[2].set_ylim(-2.75, 0.75)

    f.suptitle(f'HSC targets - priority {p} - N = {mask.sum()}')

In [0]:
pp = np.sort(obs.data['priority'][obs.data['priority'] >= 0].unique())[::-1]
for p in pp:

    f, ax = plt.subplots(1, 1)

    mask = selection & (obs.data['priority'] == p) & (~obs.data['p_member'].isna())
    bins = np.linspace(0, 1.0, 31)
    hist, _ = np.histogram(obs.data['p_member'][mask], bins=bins)
    ax.plot(0.5 * (bins[1:] + bins[:-1]), hist)

    ax.set_title(f'Priority {p}')

In [0]:
obs.get_magnitude(hsc.magnitudes['g'])

In [0]:
pp = np.sort(obs.data['priority'][obs.data['priority'] >= 0].unique())[::-1]
for p in pp:

    f, ax = plt.subplots(1, 1)

    mask = selection & (obs.data['priority'] == p) & (~obs.data['p_member'].isna())
    bins = np.linspace(17, 23, 31)
    g, _ = obs.get_magnitude(hsc.magnitudes['g'])
    hist, _ = np.histogram(g[mask], bins=bins)
    ax.plot(0.5 * (bins[1:] + bins[:-1]), hist)

    ax.set_title(f'Priority {p}')